In [ ]:
# ============================================================================
# EDA - Exploratory Data Analysis
# E-Commerce Olist Data Pipeline
# ============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime


In [ ]:
# ============================================================================
# 1. CARREGAMENTO DOS DADOS
# ============================================================================

path = '../dataset/'

print("Carregando datasets...")
df_customers = pd.read_csv(f'{path}olist_customers_dataset.csv')
df_geolocation = pd.read_csv(f'{path}olist_geolocation_dataset.csv')
df_order_items = pd.read_csv(f'{path}olist_order_items_dataset.csv')
df_order_payments = pd.read_csv(f'{path}olist_order_payments_dataset.csv')
df_order_reviews = pd.read_csv(f'{path}olist_order_reviews_dataset.csv')
df_orders = pd.read_csv(f'{path}olist_orders_dataset.csv')
df_products = pd.read_csv(f'{path}olist_products_dataset.csv')
df_sellers = pd.read_csv(f'{path}olist_sellers_dataset.csv')
df_category_translation = pd.read_csv(f'{path}product_category_name_translation.csv')

print("✓ Datasets carregados com sucesso!\n")

# Dicionário para facilitar acesso
datasets = {
    'customers': df_customers,
    'geolocation': df_geolocation,
    'order_items': df_order_items,
    'order_payments': df_order_payments,
    'order_reviews': df_order_reviews,
    'orders': df_orders,
    'products': df_products,
    'sellers': df_sellers,
    'category_translation': df_category_translation
}

In [ ]:
# ============================================================================
# 2. VISÃO GERAL DOS DATASETS
# ============================================================================

print("=" * 50)
print("RESUMO GERAL DOS DATASETS")
print("=" * 50)

summary_data = []

for name, df in datasets.items():
    summary_data.append({
        'index': len(summary_data) + 1,
        'Dataset': name,
        'Linhas': len(df),
        'Colunas': len(df.columns),
        'Memória (MB)': round(df.memory_usage(deep=True).sum() / 1024**2, 2),
        'Valores Faltantes': df.isnull().sum().sum(),
        'Duplicatas': df.duplicated().sum()
    })

df_summary = pd.DataFrame(summary_data)
print(df_summary.to_string(index=False))
print("\n")

# order_reviews tem 145903 valores faltantes -> clientes podem ou não fazer review
# orders tem 4908 valores faltantes -> Pedidos com valores faltantes
# products tem 2448 valores faltantes -> Produtos cadastrados com atributos faltantes

In [ ]:
# ============================================================================
# 3. ANÁLISE DE QUALIDADE DE DADOS - VALORES FALTANTES
# ============================================================================

print("=" * 80)
print("ANÁLISE DE VALORES FALTANTES POR DATASET")
print("=" * 80)

for name, df in datasets.items():
    missing = df.isnull().sum()
    missing_pct = (missing / len(df) * 100).round(2)
    
    missing_df = pd.DataFrame({
        'Coluna': missing.index,
        'Valores Faltantes': missing.values,
        'Percentual (%)': missing_pct.values
    })
    missing_df = missing_df[missing_df['Valores Faltantes'] > 0].sort_values('Valores Faltantes', ascending=False)
    
    if len(missing_df) > 0:
        print(f"\n{name.upper()}:")
        print(missing_df.to_string(index=False))
    else:
        print(f"\n{name.upper()}: ✓ Sem valores faltantes")

print("\n")

In [ ]:
# ============================================================================
# 4. ANÁLISE DO DATASET DE PEDIDOS (PRINCIPAL)
# ============================================================================

print("=" * 80)
print("ANÁLISE DO DATASET DE PEDIDOS")
print("=" * 80)

# Informações básicas
print(f"\nTotal de pedidos: {len(df_orders):,}")
print(f"Período: {df_orders['order_purchase_timestamp'].min()} até {df_orders['order_purchase_timestamp'].max()}")

# Conversão de datas
date_cols = [col for col in df_orders.columns if 'timestamp' in col or 'date' in col]
for col in date_cols:
    df_orders[col] = pd.to_datetime(df_orders[col], errors='coerce')

# Status dos pedidos
print("\n--- Distribuição de Status dos Pedidos ---")
status_dist = df_orders['order_status'].value_counts()
print(status_dist)
print(f"\nPercentual:")
print((status_dist / len(df_orders) * 100).round(2))

# Análise temporal
df_orders['order_year_month'] = df_orders['order_purchase_timestamp'].dt.to_period('M')
orders_by_month = df_orders.groupby('order_year_month').size()

print("\n--- Pedidos por Mês ---")
print(orders_by_month.tail(12))

In [ ]:
# ============================================================================
# 5. ANÁLISE DE CLIENTES
# ============================================================================

print("=" * 80)
print("ANÁLISE DE CLIENTES")
print("=" * 80)

print(f"\nTotal de registros de clientes: {len(df_customers):,}")
print(f"Total de clientes únicos: {df_customers['customer_unique_id'].nunique():,}")
print(f"Total de customer_id: {df_customers['customer_id'].nunique():,}")

# Clientes por estado
print("\n--- Top 10 Estados por Número de Clientes ---")
customers_by_state = df_customers['customer_state'].value_counts().head(10)
print(customers_by_state)

# Clientes por cidade
print("\n--- Top 10 Cidades por Número de Clientes ---")
customers_by_city = df_customers['customer_city'].value_counts().head(10)
print(customers_by_city)

# Análise de recorrência (clientes que compraram mais de uma vez)
customer_order_count = df_orders.merge(df_customers, on='customer_id', how='left')
customer_frequency = customer_order_count.groupby('customer_unique_id')['order_id'].count()
recurring_customers = (customer_frequency > 1).sum()

print(f"\nClientes recorrentes (compraram mais de 1 vez): {recurring_customers:,}")
print(f"Percentual de clientes recorrentes: {(recurring_customers / df_customers['customer_unique_id'].nunique() * 100):.2f}%")

In [ ]:
# ============================================================================
# 6. ANÁLISE DE PRODUTOS
# ============================================================================

print("=" * 80)
print("ANÁLISE DE PRODUTOS")
print("=" * 80)

print(f"\nTotal de produtos: {df_products['product_id'].nunique():,}")
print(f"Total de categorias: {df_products['product_category_name'].nunique():,}")

# Merge com tradução de categorias
df_products_enriched = df_products.merge(
    df_category_translation, 
    on='product_category_name', 
    how='left'
)

# Top categorias
print("\n--- Top 15 Categorias de Produtos ---")
top_categories = df_products_enriched['product_category_name_english'].value_counts().head(15)
print(top_categories)

# Análise de dimensões e peso
print("\n--- Estatísticas de Dimensões e Peso dos Produtos ---")
dimension_cols = ['product_length_cm', 'product_height_cm', 'product_width_cm', 'product_weight_g']
for col in dimension_cols:
    if col in df_products.columns:
        missing_count = df_products[col].isnull().sum()
        missing_pct = (missing_count / len(df_products) * 100)
        print(f"\n{col}:")
        print(f"  Média: {df_products[col].mean():.2f}")
        print(f"  Mediana: {df_products[col].median():.2f}")
        print(f"  Min: {df_products[col].min():.2f}")
        print(f"  Max: {df_products[col].max():.2f}")
        print(f"  Valores faltantes: {missing_count:,} ({missing_pct:.2f}%)")

In [ ]:
# ============================================================================
# 7. ANÁLISE DE ITENS DE PEDIDOS E VALORES
# ============================================================================

print("=" * 80)
print("ANÁLISE DE ITENS DE PEDIDOS E VALORES")
print("=" * 80)

print(f"\nTotal de itens de pedidos: {len(df_order_items):,}")
print(f"Total de pedidos únicos: {df_order_items['order_id'].nunique():,}")
print(f"Total de produtos únicos: {df_order_items['product_id'].nunique():,}")
print(f"Total de vendedores únicos: {df_order_items['seller_id'].nunique():,}")

# Estatísticas de preço e frete
print("\n--- Estatísticas de Preço ---")
print(df_order_items['price'].describe())

print("\n--- Estatísticas de Frete ---")
print(df_order_items['freight_value'].describe())

# Valor total por pedido
order_totals = df_order_items.groupby('order_id').agg({
    'price': 'sum',
    'freight_value': 'sum'
}).reset_index()
order_totals['total_value'] = order_totals['price'] + order_totals['freight_value']

print("\n--- Estatísticas de Valor Total por Pedido ---")
print(order_totals['total_value'].describe())

# Itens por pedido
items_per_order = df_order_items.groupby('order_id').size()
print("\n--- Estatísticas de Itens por Pedido ---")
print(f"Média: {items_per_order.mean():.2f}")
print(f"Mediana: {items_per_order.median():.0f}")
print(f"Min: {items_per_order.min()}")
print(f"Max: {items_per_order.max()}")

In [ ]:
# ============================================================================
# 8. ANÁLISE DE PAGAMENTOS
# ============================================================================

print("=" * 80)
print("ANÁLISE DE PAGAMENTOS")
print("=" * 80)

print(f"\nTotal de registros de pagamento: {len(df_order_payments):,}")
print(f"Total de pedidos únicos com pagamento: {df_order_payments['order_id'].nunique():,}")

# Tipos de pagamento
print("\n--- Distribuição por Tipo de Pagamento ---")
payment_types = df_order_payments['payment_type'].value_counts()
print(payment_types)
print(f"\nPercentual:")
print((payment_types / len(df_order_payments) * 100).round(2))

# Estatísticas de valor de pagamento
print("\n--- Estatísticas de Valor de Pagamento ---")
print(df_order_payments['payment_value'].describe())

# Pedidos com múltiplas formas de pagamento
payments_per_order = df_order_payments.groupby('order_id').size()
multiple_payments = (payments_per_order > 1).sum()
print(f"\nPedidos com múltiplas formas de pagamento: {multiple_payments:,} ({(multiple_payments/len(payments_per_order)*100):.2f}%)")

# Análise de parcelas
print("\n--- Distribuição de Parcelas ---")
installments_dist = df_order_payments['payment_installments'].value_counts().sort_index()
print(installments_dist.head(15))

In [ ]:
# ============================================================================
# 9. ANÁLISE DE AVALIAÇÕES (REVIEWS)
# ============================================================================

print("=" * 80)
print("ANÁLISE DE AVALIAÇÕES")
print("=" * 80)

print(f"\nTotal de avaliações: {len(df_order_reviews):,}")
print(f"Total de pedidos avaliados: {df_order_reviews['order_id'].nunique():,}")

# Distribuição de notas
print("\n--- Distribuição de Notas (review_score) ---")
score_dist = df_order_reviews['review_score'].value_counts().sort_index()
print(score_dist)
print(f"\nPercentual:")
print((score_dist / len(df_order_reviews) * 100).round(2))

# Estatísticas
print(f"\nMédia de notas: {df_order_reviews['review_score'].mean():.2f}")
print(f"Mediana de notas: {df_order_reviews['review_score'].median():.0f}")

# Análise de comentários
df_order_reviews['has_comment'] = df_order_reviews['review_comment_message'].notna()
print(f"\nAvaliações com comentário: {df_order_reviews['has_comment'].sum():,} ({(df_order_reviews['has_comment'].sum()/len(df_order_reviews)*100):.2f}%)")

# Relação entre nota e comentário
if 'review_comment_message' in df_order_reviews.columns:
    print("\n--- Média de Notas por Presença de Comentário ---")
    score_by_comment = df_order_reviews.groupby('has_comment')['review_score'].mean()
    print(score_by_comment)

In [ ]:
# ============================================================================
# 10. ANÁLISE DE VENDEDORES
# ============================================================================

print("=" * 80)
print("ANÁLISE DE VENDEDORES")
print("=" * 80)

print(f"\nTotal de vendedores: {len(df_sellers):,}")

# Vendedores por estado
print("\n--- Top 10 Estados por Número de Vendedores ---")
sellers_by_state = df_sellers['seller_state'].value_counts().head(10)
print(sellers_by_state)

# Vendedores por cidade
print("\n--- Top 10 Cidades por Número de Vendedores ---")
sellers_by_city = df_sellers['seller_city'].value_counts().head(10)
print(sellers_by_city)

# Análise de performance de vendedores
seller_performance = df_order_items.groupby('seller_id').agg({
    'order_id': 'nunique',
    'price': 'sum',
    'freight_value': 'sum'
}).reset_index()
seller_performance.columns = ['seller_id', 'total_orders', 'total_revenue', 'total_freight']
seller_performance['total_value'] = seller_performance['total_revenue'] + seller_performance['total_freight']

print("\n--- Top 10 Vendedores por Receita Total ---")
top_sellers = seller_performance.nlargest(10, 'total_value')[['seller_id', 'total_orders', 'total_value']]
print(top_sellers)

In [ ]:
# ============================================================================
# 11. ANÁLISE DE RELACIONAMENTOS ENTRE DATASETS
# ============================================================================

print("=" * 80)
print("ANÁLISE DE RELACIONAMENTOS E INTEGRIDADE")
print("=" * 80)

# Verificação de chaves primárias e relacionamentos
print("\n--- Verificação de Integridade Referencial ---")

# Orders vs Customers
orders_with_customers = df_orders['customer_id'].isin(df_customers['customer_id']).sum()
print(f"Pedidos com customer_id válido: {orders_with_customers:,} / {len(df_orders):,} ({(orders_with_customers/len(df_orders)*100):.2f}%)")

# Order Items vs Orders
order_items_with_orders = df_order_items['order_id'].isin(df_orders['order_id']).sum()
print(f"Itens com order_id válido: {order_items_with_orders:,} / {len(df_order_items):,} ({(order_items_with_orders/len(df_order_items)*100):.2f}%)")

# Order Items vs Products
order_items_with_products = df_order_items['product_id'].isin(df_products['product_id']).sum()
print(f"Itens com product_id válido: {order_items_with_products:,} / {len(df_order_items):,} ({(order_items_with_products/len(df_order_items)*100):.2f}%)")

# Order Items vs Sellers
order_items_with_sellers = df_order_items['seller_id'].isin(df_sellers['seller_id']).sum()
print(f"Itens com seller_id válido: {order_items_with_sellers:,} / {len(df_order_items):,} ({(order_items_with_sellers/len(df_order_items)*100):.2f}%)")

# Order Payments vs Orders
payments_with_orders = df_order_payments['order_id'].isin(df_orders['order_id']).sum()
print(f"Pagamentos com order_id válido: {payments_with_orders:,} / {len(df_order_payments):,} ({(payments_with_orders/len(df_order_payments)*100):.2f}%)")

# Order Reviews vs Orders
reviews_with_orders = df_order_reviews['order_id'].isin(df_orders['order_id']).sum()
print(f"Avaliações com order_id válido: {reviews_with_orders:,} / {len(df_order_reviews):,} ({(reviews_with_orders/len(df_order_reviews)*100):.2f}%)")

In [ ]:
# ============================================================================
# 12. ANÁLISE TEMPORAL DETALHADA
# ============================================================================

print("=" * 80)
print("ANÁLISE TEMPORAL")
print("=" * 80)

# Garantir que as datas estão convertidas
if 'order_purchase_timestamp' in df_orders.columns:
    df_orders['order_purchase_timestamp'] = pd.to_datetime(df_orders['order_purchase_timestamp'], errors='coerce')
    
    # Análise por ano
    df_orders['order_year'] = df_orders['order_purchase_timestamp'].dt.year
    orders_by_year = df_orders.groupby('order_year').size()
    print("\n--- Pedidos por Ano ---")
    print(orders_by_year)
    
    # Análise por mês
    df_orders['order_month'] = df_orders['order_purchase_timestamp'].dt.month
    orders_by_month = df_orders.groupby('order_month').size()
    print("\n--- Pedidos por Mês (Média) ---")
    print(orders_by_month)
    
    # Análise por dia da semana
    df_orders['order_day_of_week'] = df_orders['order_purchase_timestamp'].dt.day_name()
    orders_by_dow = df_orders.groupby('order_day_of_week').size()
    print("\n--- Pedidos por Dia da Semana ---")
    print(orders_by_dow)
    
    # Tendência mensal
    df_orders['order_year_month'] = df_orders['order_purchase_timestamp'].dt.to_period('M')
    monthly_trend = df_orders.groupby('order_year_month').size()
    print("\n--- Tendência Mensal (Últimos 12 meses) ---")
    print(monthly_trend.tail(12))

In [ ]:
# ============================================================================
# 13. ANÁLISE DE GEOGRAFIA
# ============================================================================

print("=" * 80)
print("ANÁLISE GEOGRÁFICA")
print("=" * 80)

# Estados com mais pedidos
customer_orders = df_orders.merge(df_customers, on='customer_id', how='left')
orders_by_state = customer_orders['customer_state'].value_counts()

print("\n--- Top 10 Estados por Número de Pedidos ---")
print(orders_by_state.head(10))

# Análise de geolocalização
print(f"\nTotal de registros de geolocalização: {len(df_geolocation):,}")
print(f"CEPs únicos: {df_geolocation['geolocation_zip_code_prefix'].nunique():,}")

# Estados na geolocalização
print("\n--- Top 10 Estados na Base de Geolocalização ---")
geo_by_state = df_geolocation['geolocation_state'].value_counts().head(10)
print(geo_by_state)

# Validação de coordenadas
valid_coords = df_geolocation[
    (df_geolocation['geolocation_lat'].between(-90, 90)) & 
    (df_geolocation['geolocation_lng'].between(-180, 180))
]
print(f"\nCoordenadas válidas: {len(valid_coords):,} / {len(df_geolocation):,} ({(len(valid_coords)/len(df_geolocation)*100):.2f}%)")

In [ ]:
# ============================================================================
# 14. VISUALIZAÇÕES - DISTRIBUIÇÕES E TENDÊNCIAS
# ============================================================================

# Configuração de figuras
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Análises Visuais Principais', fontsize=16, fontweight='bold')

# 1. Distribuição de Status de Pedidos
ax1 = axes[0, 0]
status_counts = df_orders['order_status'].value_counts()
ax1.barh(status_counts.index, status_counts.values, color='steelblue')
ax1.set_xlabel('Quantidade de Pedidos')
ax1.set_ylabel('Status')
ax1.set_title('Distribuição de Status dos Pedidos')
ax1.grid(axis='x', alpha=0.3)

# 2. Distribuição de Notas de Avaliação
ax2 = axes[0, 1]
score_counts = df_order_reviews['review_score'].value_counts().sort_index()
ax2.bar(score_counts.index, score_counts.values, color='coral', edgecolor='black')
ax2.set_xlabel('Nota')
ax2.set_ylabel('Quantidade de Avaliações')
ax2.set_title('Distribuição de Notas de Avaliação')
ax2.set_xticks(range(1, 6))
ax2.grid(axis='y', alpha=0.3)

# 3. Tipos de Pagamento
ax3 = axes[1, 0]
payment_counts = df_order_payments['payment_type'].value_counts()
ax3.pie(payment_counts.values, labels=payment_counts.index, autopct='%1.1f%%', startangle=90)
ax3.set_title('Distribuição de Tipos de Pagamento')

# 4. Tendência Mensal de Pedidos
ax4 = axes[1, 1]
if 'order_purchase_timestamp' in df_orders.columns:
    df_orders['order_year_month'] = pd.to_datetime(df_orders['order_purchase_timestamp']).dt.to_period('M')
    monthly_orders = df_orders.groupby('order_year_month').size()
    monthly_orders.index = monthly_orders.index.astype(str)
    ax4.plot(monthly_orders.index, monthly_orders.values, marker='o', linewidth=2, markersize=4)
    ax4.set_xlabel('Mês')
    ax4.set_ylabel('Quantidade de Pedidos')
    ax4.set_title('Tendência Mensal de Pedidos')
    ax4.tick_params(axis='x', rotation=45)
    ax4.grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# ============================================================================
# 15. VISUALIZAÇÕES - ANÁLISES DE VALORES E PERFORMANCE
# ============================================================================

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Análises de Valores e Performance', fontsize=16, fontweight='bold')

# 1. Distribuição de Preços
ax1 = axes[0, 0]
# Remover outliers para melhor visualização (top 1%)
price_99 = df_order_items['price'].quantile(0.99)
price_filtered = df_order_items[df_order_items['price'] <= price_99]['price']
ax1.hist(price_filtered, bins=50, color='skyblue', edgecolor='black', alpha=0.7)
ax1.set_xlabel('Preço (R$)')
ax1.set_ylabel('Frequência')
ax1.set_title('Distribuição de Preços (até percentil 99)')
ax1.grid(alpha=0.3)

# 2. Top 10 Categorias de Produtos
ax2 = axes[0, 1]
if 'product_category_name_english' in df_products_enriched.columns:
    top_cats = df_products_enriched['product_category_name_english'].value_counts().head(10)
    ax2.barh(range(len(top_cats)), top_cats.values, color='lightgreen')
    ax2.set_yticks(range(len(top_cats)))
    ax2.set_yticklabels(top_cats.index, fontsize=9)
    ax2.set_xlabel('Quantidade de Produtos')
    ax2.set_title('Top 10 Categorias de Produtos')
    ax2.invert_yaxis()
    ax2.grid(axis='x', alpha=0.3)

# 3. Top 10 Estados por Pedidos
ax3 = axes[1, 0]
customer_orders = df_orders.merge(df_customers, on='customer_id', how='left')
top_states = customer_orders['customer_state'].value_counts().head(10)
ax3.barh(range(len(top_states)), top_states.values, color='salmon')
ax3.set_yticks(range(len(top_states)))
ax3.set_yticklabels(top_states.index)
ax3.set_xlabel('Quantidade de Pedidos')
ax3.set_title('Top 10 Estados por Número de Pedidos')
ax3.invert_yaxis()
ax3.grid(axis='x', alpha=0.3)

# 4. Distribuição de Parcelas
ax4 = axes[1, 1]
installments_dist = df_order_payments['payment_installments'].value_counts().sort_index().head(15)
ax4.bar(installments_dist.index, installments_dist.values, color='plum', edgecolor='black')
ax4.set_xlabel('Número de Parcelas')
ax4.set_ylabel('Quantidade de Pagamentos')
ax4.set_title('Distribuição de Parcelas')
ax4.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# ============================================================================
# 16. ANÁLISE DE CORRELAÇÕES E RELACIONAMENTOS
# ============================================================================

# Criar dataset consolidado para análise
print("=" * 80)
print("ANÁLISE DE CORRELAÇÕES")
print("=" * 80)

# Merge principal: orders + order_items + payments
df_consolidated = df_orders.merge(
    df_order_items.groupby('order_id').agg({
        'price': 'sum',
        'freight_value': 'sum',
        'product_id': 'count'
    }).reset_index().rename(columns={'product_id': 'items_count'}),
    on='order_id',
    how='left'
)

df_consolidated = df_consolidated.merge(
    df_order_payments.groupby('order_id')['payment_value'].sum().reset_index(),
    on='order_id',
    how='left'
)

df_consolidated = df_consolidated.merge(
    df_order_reviews[['order_id', 'review_score']],
    on='order_id',
    how='left'
)

# Calcular tempo de entrega (se disponível)
if 'order_delivered_customer_date' in df_consolidated.columns and 'order_purchase_timestamp' in df_consolidated.columns:
    df_consolidated['order_purchase_timestamp'] = pd.to_datetime(df_consolidated['order_purchase_timestamp'])
    df_consolidated['order_delivered_customer_date'] = pd.to_datetime(df_consolidated['order_delivered_customer_date'])
    df_consolidated['delivery_time_days'] = (
        df_consolidated['order_delivered_customer_date'] - df_consolidated['order_purchase_timestamp']
    ).dt.days

# Selecionar colunas numéricas para correlação
numeric_cols = ['price', 'freight_value', 'items_count', 'payment_value', 'review_score']
if 'delivery_time_days' in df_consolidated.columns:
    numeric_cols.append('delivery_time_days')

corr_cols = [col for col in numeric_cols if col in df_consolidated.columns]
df_corr = df_consolidated[corr_cols].corr()

print("\n--- Matriz de Correlação ---")
print(df_corr.round(3))

# Visualização da matriz de correlação
plt.figure(figsize=(10, 8))
sns.heatmap(df_corr, annot=True, fmt='.2f', cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Matriz de Correlação - Variáveis Numéricas', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================================
# 17. INSIGHTS E CONCLUSÕES
# ============================================================================

print("=" * 80)
print("PRINCIPAIS INSIGHTS E CONCLUSÕES")
print("=" * 80)

insights = []

# 1. Volume de dados
total_orders = len(df_orders)
total_customers = df_customers['customer_unique_id'].nunique()
total_products = df_products['product_id'].nunique()
total_revenue = df_order_items['price'].sum() + df_order_items['freight_value'].sum()

insights.append(f"1. VOLUME: {total_orders:,} pedidos de {total_customers:,} clientes únicos")
insights.append(f"   - {total_products:,} produtos únicos")
insights.append(f"   - Receita total: R$ {total_revenue:,.2f}")

# 2. Qualidade dos dados
missing_total = sum([df.isnull().sum().sum() for df in datasets.values()])
insights.append(f"\n2. QUALIDADE: {missing_total:,} valores faltantes no total")

# 3. Status de pedidos
delivered_pct = (df_orders['order_status'] == 'delivered').sum() / len(df_orders) * 100
insights.append(f"\n3. PEDIDOS: {delivered_pct:.1f}% dos pedidos foram entregues")

# 4. Satisfação do cliente
avg_score = df_order_reviews['review_score'].mean()
insights.append(f"\n4. SATISFAÇÃO: Nota média de {avg_score:.2f}/5.0")
high_satisfaction = (df_order_reviews['review_score'] >= 4).sum() / len(df_order_reviews) * 100
insights.append(f"   - {high_satisfaction:.1f}% das avaliações são 4 ou 5 estrelas")

# 5. Pagamentos
credit_card_pct = (df_order_payments['payment_type'] == 'credit_card').sum() / len(df_order_payments) * 100
insights.append(f"\n5. PAGAMENTOS: {credit_card_pct:.1f}% dos pagamentos são por cartão de crédito")

# 6. Geografia
top_state = customer_orders['customer_state'].value_counts().index[0]
top_state_pct = customer_orders['customer_state'].value_counts().iloc[0] / len(customer_orders) * 100
insights.append(f"\n6. GEOGRAFIA: {top_state} concentra {top_state_pct:.1f}% dos pedidos")

# 7. Recorrência
insights.append(f"\n7. RECORRÊNCIA: {recurring_customers:,} clientes fizeram mais de 1 compra")

# 8. Produtos
top_category = df_products_enriched['product_category_name_english'].value_counts().index[0] if 'product_category_name_english' in df_products_enriched.columns else "N/A"
insights.append(f"\n8. PRODUTOS: Categoria mais vendida: {top_category}")

for insight in insights:
    print(insight)